In [6]:
sp_dc = "AQApodUziie66zxTTHXzWdn2NzPU2QJupuYnQyzAd6UFebPJCwHD0n1CEvMqwKd_sEkuwyn-IiWcJY9JKD5agnBvwaMyI7G9VGacRNS-a2FNfXqmPFpYQMBgywIBZ-FlsNqAHqd6HQhZeWYniUdjSKmx7A2nN7jW"

In [7]:
from syrics.api import Spotify

In [8]:
sp = Spotify(sp_dc)

In [13]:
sp.get_lyrics("0AAMnNeIc6CdnfNU85GwCH")

{'lyrics': {'syncType': 'LINE_SYNCED',
  'lines': [{'startTimeMs': '20',
    'words': '(Metro)',
    'syllables': [],
    'endTimeMs': '0'},
   {'startTimeMs': '2590', 'words': 'Yeah', 'syllables': [], 'endTimeMs': '0'},
   {'startTimeMs': '5460', 'words': 'Hmm', 'syllables': [], 'endTimeMs': '0'},
   {'startTimeMs': '7820',
    'words': "Oh my, she's a long way from suburban towns",
    'syllables': [],
    'endTimeMs': '0'},
   {'startTimeMs': '11930',
    'words': "Came to the city for the love, got her hurtin' now",
    'syllables': [],
    'endTimeMs': '0'},
   {'startTimeMs': '15850',
    'words': "Oh my, she's a long way from suburban town",
    'syllables': [],
    'endTimeMs': '0'},
   {'startTimeMs': '19800',
    'words': 'Long way, really long way from suburbia',
    'syllables': [],
    'endTimeMs': '0'},
   {'startTimeMs': '23810',
    'words': 'Small town love, fall down love, not medicated',
    'syllables': [],
    'endTimeMs': '0'},
   {'startTimeMs': '27940',
    'wor

In [10]:
print(sp)